# **Классификация позитивных комментариев по работе управляющей компании с использованием RuRobertaa**

##**Что было сделано:**

**Подготовка данных**

* Из отзывов выделила пять основных категорий и отметила, какие категории относятся к каждому отзыву:

  * Нравится скорость отработки заявок
  * Нравится качество выполнения заявки
  * Нравится качество работы сотрудников
  * Понравилось выполнение заявки
  * Вопрос решен

**Обработка текста**

* Перевёла текст отзывов в числовые векторы (эмбеддинги) с помощью модели RuRoberta, чтобы модель могла понять смысл предложений целиком, а не только отдельные слова.

**Обучение модели**

* На основе полученных векторов обучила модель логистической регрессии отдельно для каждой категории.
* Разделила данные на обучающую и тестовую части, чтобы честно оценить работу модели.

**Результаты**

* Модель показала высокое качество работы (ROC-AUC до 0.97 по большинству категорий).


### Скачиваем библиотеки

In [35]:
# ast — встроенный, устанавливать не нужно
%pip install pandas numpy scikit-learn sentence-transformers


### Необходимые библиотеки

In [36]:
import pandas as pd
import re
import ast
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score
from collections import Counter
import numpy as np

### Загрузка и предварительная обработка данных

In [37]:
# Загрузка данных
df = pd.read_csv("project-1-at-2025-05-17-22-04-8c91f7a3.csv")
df.head()

,annotation_id,annotator,categories,comment,created_at,id,lead_time,rating,updated_at,Вопрос решен,Нравится качество выполнения заявки,Нравится качество работы сотрудников,Нравится скорость отработки заявок,Понравилось выполнение заявки
0,1,1,Вопрос решен,спасибо,2025-05-16T11:08:48.994749Z,2945792,11.009,5,2025-05-16T11:08:48.994749Z,0,0,0,0,0
1,2,1,Вопрос решен,отлично,2025-05-16T11:09:09.137671Z,3380332,19.703,5,2025-05-16T11:09:09.137671Z,0,0,0,0,0
2,3,1,Нравится скорость отработки заявок,благодарю за оперативное решение проблемы,2025-05-16T11:09:17.579820Z,3381812,8.067,5,2025-05-16T11:09:17.579820Z,0,0,0,0,0
3,4,1,Нравится качество работы сотрудников,прекрасный специалист побольше таких,2025-05-16T11:09:30.270300Z,3461991,12.262,5,2025-05-16T11:09:30.270300Z,0,0,0,0,0
4,5,1,Другое,пересчет и скорость,2025-05-16T11:09:42.105464Z,3572768,11.454,1,2025-05-16T11:09:42.105464Z,0,0,0,0,0


### Парсинг и разметка категорий

In [38]:
category_cols = [
    'Нравится скорость отработки заявок',
    'Нравится качество выполнения заявки',
    'Нравится качество работы сотрудников',
    'Понравилось выполнение заявки',
    'Вопрос решен'
]

def extract_categories(cat_value):
    if pd.isna(cat_value):
        return []
    if isinstance(cat_value, str) and cat_value.strip().startswith("{"):
        try:
            obj = ast.literal_eval(cat_value.replace('""', '"'))
            return obj.get('choices', [])
        except Exception:
            return []
    else:
        return [cat_value.strip()]

for col in category_cols:
    df[col] = 0

for idx, row in df.iterrows():
    found = extract_categories(row['categories'])
    for col in category_cols:
        if col in found:
            df.at[idx, col] = 1

for col in category_cols:
    print(f"{col}: положительных - {df[col].sum()}, отрицательных - {len(df) - df[col].sum()}")

Нравится скорость отработки заявок: положительных - 605, отрицательных - 918
Нравится качество выполнения заявки: положительных - 139, отрицательных - 1384
Нравится качество работы сотрудников: положительных - 374, отрицательных - 1149
Понравилось выполнение заявки: положительных - 213, отрицательных - 1310
Вопрос решен: положительных - 295, отрицательных - 1228


### Векторизация текста с помощью RuRoberta

In [39]:
model = SentenceTransformer('DeepPavlov/rubert-base-cased-sentence')
X = model.encode(df['comment'].astype(str).tolist(), show_progress_bar=True)

Batches:   0%|          | 0/48 [00:00<?, ?it/s]

### Разделение данных

In [40]:
results = {}

for cat in category_cols:
    y = df[cat]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )
    if len(np.unique(y_train)) < 2 or len(np.unique(y_test)) < 2:
        print(f"Пропускаю категорию '{cat}': только один класс после разбиения!")
        continue
    clf = LogisticRegression(max_iter=1000, class_weight='balanced')
    clf.fit(X_train, y_train)

    y_prob = clf.predict_proba(X_test)[:, 1]
    y_pred = clf.predict(X_test)

    roc = roc_auc_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    results[cat] = {'roc_auc': roc, 'f1': f1}
    print(f"{cat}: ROC-AUC = {roc:.3f}, F1 = {f1:.3f}")

print("\nИтоговые метрики по категориям:")
for cat, mets in results.items():
    print(f"{cat}: ROC-AUC = {mets['roc_auc']:.3f}, F1 = {mets['f1']:.3f}")

Нравится скорость отработки заявок: ROC-AUC = 0.890, F1 = 0.869
Нравится качество выполнения заявки: ROC-AUC = 0.749, F1 = 0.453
Нравится качество работы сотрудников: ROC-AUC = 0.940, F1 = 0.913
Понравилось выполнение заявки: ROC-AUC = 0.682, F1 = 0.414
Вопрос решен: ROC-AUC = 0.737, F1 = 0.547

Итоговые метрики по категориям:
Нравится скорость отработки заявок: ROC-AUC = 0.890, F1 = 0.869
Нравится качество выполнения заявки: ROC-AUC = 0.749, F1 = 0.453
Нравится качество работы сотрудников: ROC-AUC = 0.940, F1 = 0.913
Понравилось выполнение заявки: ROC-AUC = 0.682, F1 = 0.414
Вопрос решен: ROC-AUC = 0.737, F1 = 0.547


### Анализ частоты слов

In [41]:
def common_words_analysis(df, category):
    comments = df[df[category] == 1]['comment'].str.lower().apply(lambda x: re.sub(r"[^а-яёa-z\s]", "", x))
    all_words = ' '.join(comments).split()
    top_words = Counter(all_words).most_common(10)
    print(f"\nТоп-10 самых частых слов в классе '{category}':")
    for word, count in top_words:
        print(f"{word}: {count} раз")

for cat in category_cols:
    common_words_analysis(df, cat)


Топ-10 самых частых слов в классе 'Нравится скорость отработки заявок':
спасибо: 342 раз
быстро: 304 раз
и: 209 раз
оперативно: 186 раз
за: 142 раз
все: 106 раз
очень: 94 раз
большое: 82 раз
в: 69 раз
качественно: 66 раз

Топ-10 самых частых слов в классе 'Нравится качество выполнения заявки':
и: 98 раз
спасибо: 75 раз
качественно: 70 раз
быстро: 62 раз
за: 30 раз
все: 30 раз
оперативно: 24 раз
большое: 23 раз
очень: 19 раз
работу: 15 раз

Топ-10 самых частых слов в классе 'Нравится качество работы сотрудников':
спасибо: 214 раз
и: 127 раз
все: 95 раз
быстро: 87 раз
мастер: 78 раз
за: 71 раз
очень: 67 раз
в: 61 раз
большое: 54 раз
оперативно: 40 раз

Топ-10 самых частых слов в классе 'Понравилось выполнение заявки':
спасибо: 149 раз
все: 55 раз
за: 54 раз
и: 54 раз
большое: 52 раз
всё: 27 раз
очень: 24 раз
быстро: 23 раз
отлично: 22 раз
работу: 19 раз

Топ-10 самых частых слов в классе 'Вопрос решен':
спасибо: 141 раз
за: 42 раз
все: 40 раз
в: 36 раз
большое: 28 раз
всё: 25 раз
и: 23 

### Сохранение размеченного датасета

In [42]:
df.to_csv("dataset_with_labels.csv", index=False)